In [1]:
import numpy as np
import h5py
from Code.transform import transforms

In [2]:
transforms

{'ta': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'ua': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'va': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'wap': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'hus': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'zeta': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'zg': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'd': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'ps': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>]}

In [20]:
with h5py.File('./data/11y1burn6hres45mstep_mini/chan81_ncdf4.nc', 'a') as raw:
    print([k for k in raw['Transformed'].attrs])

[]


In [40]:
redo_flag = True
from time import time,ctime
with h5py.File('./data/11y1burn6hres45mstep_mini/chan81_ncdf4.nc', 'a') as raw:
    #with h5py.File('TransformedDS.hdf5','a') as file:
    if "Transformed" not in raw:
        raw.create_dataset("Transformed",(14608,81,32,64),dtype = 'f4')
    if redo_flag:
        descr = [          
             ('ta',  'air_temperature',       '10','130','K'),
             ('ua',  'eastward_wind',         '10','131','m s-1'),
             ('va',  'northward_wind',        '10','132','m s-1'),
             ('hus', 'specific_humidity',     '10','133','kg/kg'),
             ('ps',  'surface_air_pressure',  '1 ','134','hPa'),
             ('wap',  'vertical_air_velocity', '10','135','Pa s-1'),
             ('zeta','atm_relative_vorticity','10','138','s-1'),
             ('d',   'divergence of wind',    '10','155','s-1'),
             ('zg',  'geopotential_height',   '10','156','m'),
        ]
        cumlv = 0
        transform_time_by_lv = []
        for k,_,lv,_,_ in descr:
            
            d = raw[k]
            if k == 'ps':
                d = np.expand_dims(d,1)
            transform_meta = []
            for l in range(int(lv)):
                s_time = time()
                print("reached",k,l)
                transformed, attrs = transforms[k][0](d[:,l,:,:])
                transform_meta.append([[k,str(v)] for k,v in attrs.items()])
                raw['Transformed'][:,cumlv,:,:] = transformed
                cumlv += 1
                
                e_time = time()
                taken = e_time-s_time
                transform_time_by_lv.append(taken)
                print("Done ",k,l,'seconds:',int(taken),"finished:",ctime(e_time))
            raw['Transformed'].attrs.create(
                name = k+" transformation meta statistic by hPa levels 100 to 1000",
                data = transform_meta
            )
            
        raw['Transformed'].attrs.create(name = "channels info in order: abbrev, name, levels, puma code, unit",
             data = descr
        )
        raw['Transformed'].attrs.create(name = "order of atmospheric level in pressure: from space to surface",
            data = [100,200,300,400,500,600,700,800,900,1000]
        )
        raw['Transformed'].attrs.create(name = "dimension info",
            data = [
                '14608 timesteps of 6h, date since 0002-01-01 00:00:00',
                '81 channels of features',
                '32 latitudes',
                '64 longitudes',
            ]
        )
    if "Latitude_map" not in raw:
        raw.create_dataset("Latitude_map",(32,64),dtype = 'f4')
        raw['Latitude_map'][:] = np.tile(np.linspace(1,-1,32),(64,1)).T 

reached ta 0
Done  ta 0 seconds: 30 finished: Sun Jun 26 15:05:42 2022
reached ta 1
Done  ta 1 seconds: 33 finished: Sun Jun 26 15:06:16 2022
reached ta 2
Done  ta 2 seconds: 24 finished: Sun Jun 26 15:06:40 2022
reached ta 3
Done  ta 3 seconds: 25 finished: Sun Jun 26 15:07:06 2022
reached ta 4
Done  ta 4 seconds: 26 finished: Sun Jun 26 15:07:32 2022
reached ta 5
Done  ta 5 seconds: 27 finished: Sun Jun 26 15:08:00 2022
reached ta 6
Done  ta 6 seconds: 28 finished: Sun Jun 26 15:08:28 2022
reached ta 7
Done  ta 7 seconds: 26 finished: Sun Jun 26 15:08:55 2022
reached ta 8
Done  ta 8 seconds: 26 finished: Sun Jun 26 15:09:21 2022
reached ta 9
Done  ta 9 seconds: 28 finished: Sun Jun 26 15:09:50 2022
reached ua 0
Done  ua 0 seconds: 27 finished: Sun Jun 26 15:10:18 2022
reached ua 1
Done  ua 1 seconds: 28 finished: Sun Jun 26 15:10:46 2022
reached ua 2
Done  ua 2 seconds: 28 finished: Sun Jun 26 15:11:15 2022
reached ua 3
Done  ua 3 seconds: 28 finished: Sun Jun 26 15:11:43 2022
reache

In [48]:
with h5py.File('./data/11y1burn6hres45mstep_mini/chan81_ncdf4.nc', 'r') as raw:
    p = [(x,raw['Transformed'].attrs[x]) for x in raw['Transformed'].attrs]
    for x in p:
        print("----------")
        print(x)

----------
('channels info in order: abbrev, name, levels, puma code, unit', array([['ta', 'air_temperature', '10', '130', 'K'],
       ['ua', 'eastward_wind', '10', '131', 'm s-1'],
       ['va', 'northward_wind', '10', '132', 'm s-1'],
       ['hus', 'specific_humidity', '10', '133', 'kg/kg'],
       ['ps', 'surface_air_pressure', '1 ', '134', 'hPa'],
       ['wap', 'vertical_air_velocity', '10', '135', 'Pa s-1'],
       ['zeta', 'atm_relative_vorticity', '10', '138', 's-1'],
       ['d', 'divergence of wind', '10', '155', 's-1'],
       ['zg', 'geopotential_height', '10', '156', 'm']], dtype=object))
----------
('d transformation meta statistic by hPa levels 100 to 1000', array([[['mean', '-7.6633615e-09'],
        ['std', '1.908378e-06']],

       [['mean', '-7.977629e-09'],
        ['std', '3.5284688e-06']],

       [['mean', '-1.5144217e-08'],
        ['std', '2.8961429e-06']],

       [['mean', '-1.27441835e-08'],
        ['std', '2.016561e-06']],

       [['mean', '-6.186638e-0

In [41]:
flags = []
from time import time

with h5py.File('./data/11y1burn6hres45mstep_mini/chan81_ncdf4.nc', 'r') as raw:
    for i in range(81):
        t = time()
        flag = np.isnan(raw['Transformed'][:,i,:,:]).any()
        flags.append(flag)
        print("done")
        print(time()-t)
print(flags)

done
10.730785846710205
done
10.60352349281311
done
11.622409105300903
done
9.200344562530518
done
9.19774580001831
done
9.862178087234497
done
9.420050144195557
done
9.16375994682312
done
8.592950344085693
done
8.243056535720825
done
7.619776010513306
done
7.408547878265381
done
7.351538896560669
done
7.387530326843262
done
7.546397924423218
done
8.105213642120361
done
8.722811460494995
done
10.926365375518799
done
11.695982217788696
done
11.005579233169556
done
11.154638528823853
done
11.586600542068481
done
11.579533100128174
done
11.000366449356079
done
9.931631326675415
done
10.529236316680908
done
9.848751783370972
done
9.332143545150757
done
9.692017793655396
done
10.597437381744385
done
11.088218927383423
done
12.266059398651123
done
10.732178211212158
done
10.433953523635864
done
9.615203619003296
done
9.960506200790405
done
9.520212411880493
done
9.409409999847412
done
8.79636836051941
done
9.426941871643066
done
10.606708288192749
done
10.10573697090149
done
10.1998617649078

In [39]:
with h5py.File('./data/11y1burn6hres45mstep_mini/chan81_ncdf4.nc', 'a') as raw:
    [print(k) for k in raw['Transformed'].attrs]

channels info in order: abbrev, name, levels, puma code, unit
d transformation meta statistic by hPa levels 100 to 1000
dimension info
hus transformation meta statistic by hPa levels 100 to 1000
order of atmospheric level in pressure: from space to surface
ps transformation meta statistic by hPa levels 100 to 1000
ta transformation meta statistic by hPa levels 100 to 1000
ua transformation meta statistic by hPa levels 100 to 1000
va transformation meta statistic by hPa levels 100 to 1000
zeta transformation meta statistic by hPa levels 100 to 1000
zg transformation meta statistic by hPa levels 100 to 1000


# Garbage

In [ ]:
with h5py.File('./data/11y1burn6hres45mstep_mini/chan81_ncdf4.nc', 'r') as raw:
    descr = [          
         ('ta',  'air_temperature',       '10','130','K'),
         ('ua',  'eastward_wind',         '10','131','m s-1'),
         ('va',  'northward_wind',        '10','132','m s-1'),
         ('hus', 'specific_humidity',     '10','133','kg/kg'),
         ('ps',  'surface_air_pressure',  '1 ','134','hPa'),
         ('wap',  'vertical_air_velocity', '10','135','Pa s-1'),
         ('zeta','atm_relative_vorticity','10','138','s-1'),
         ('d',   'divergence of wind',    '10','155','s-1'),
         ('zg',  'geopotential_height',   '10','156','m'),
    ]
    for k,_,lv,_,_ in descr:
        d = raw[k][:]
        if k == 'ps':
            d = np.expand_dims(d,1)
        transformed, attrs = transforms[k][0](d[:])
        print(k,attrs)

In [8]:
p = np.zeros((2,2,3))

In [9]:
np.expand_dims(p,1).shape

(2, 1, 2, 3)

In [10]:
np.tile(np.linspace(1,-1,32),(64,1)).T

array([[ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.93548387,  0.93548387,  0.93548387, ...,  0.93548387,
         0.93548387,  0.93548387],
       [ 0.87096774,  0.87096774,  0.87096774, ...,  0.87096774,
         0.87096774,  0.87096774],
       ...,
       [-0.87096774, -0.87096774, -0.87096774, ..., -0.87096774,
        -0.87096774, -0.87096774],
       [-0.93548387, -0.93548387, -0.93548387, ..., -0.93548387,
        -0.93548387, -0.93548387],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ]])

In [ ]:
f = h5py.File('TransformedDS.hdf5','a')

In [13]:
if 'LatitudeMap' not in f:
    f.create_dataset("LatitudeMap",(32,64),dtype = 'f4')
    f['LatitudeMap'] = np.tile(
        np.linspace(-1, 1, num=32, endpoint=True, retstep=False, dtype='f4', axis=0),
        (64,1)
    ).T

(32, 64)

In [17]:
raw = h5py.File('./data/11y1burn6hres45mstep_mini/chan81_ncdf4.nc','r')
#f.create_dataset("TransformedData",(14608,81, 32, 64),dtype = 'f4')
if "TransformedData" not in f:
    f.create_dataset("TransformedData",(14608,81, 32, 64),dtype = 'f4')
    
raw.close()
    